## Description

This challenge will focus on the similarity between two texts. Your objective is to write a program that takes as inputs two texts and uses a metric to determine how similar they are. Documents that are exactly the same should get a score of 1, and documents that don’t have any words in common should get a score of 0. Please use the samples below to develop your application.

You will have to make a number of decisions as you develop this solution:

- Do you count punctuation or only words?
- Which words should matter in the similarity comparison?
- Do you care about the ordering of words?
- What metric do you use to assign a numerical value to the similarity?
- What type of data structures should be used? (Hint: Dictionaries and lists are particularly helpful data structures that can be leveraged to calculate the similarity of two pieces of text.)

## Requirements

1. The document similarity algorithm does not need to perform well, and you don’t need to account for all edge cases. Focus on having some fun with it and producing code that we can discuss together.
1. Use the 3 sample texts provided below to develop your app. Samples 1 and 2 should be more similar than samples 1 and 3.
1. You may choose any language you like, but do not import any libraries.
1. The code, at a minimum, must run. Please provide clear instructions on how to run it.

When complete, please upload your codebase to a public Git repo (GitHub, Bitbucket, etc.) and email us the link. Please double-check this is publicly accessible.
Please assume the evaluator does not have prior experience executing programs in your chosen language. Therefore, please include any documentation necessary to accomplish the above requirements.

### Do you count punctuation or only words?

I only care about words. Documents will be stripped of punctuation and converted to lower case.

### Which words should matter in the similarity comparison?

I will remove some common stop words, like articles, conjunctions, and some prepositions in order to focus more on the words that provide contextual meaning to each document.

### Do you care about the ordering of words?

I started with the assumption that word ordering would be important, but the method I ended with (frequency of words in a document) does not take order into account. Including sequences of words as tokens in the way I did decreases the similarty metric because it increases the number of opportunities for the documents to differ.

### What metric do you use to assign a numerical value to the similarity?

I used cosine similarity of term frequency between each document to calculate the similarity.

### What type of data structures should be used?

I will use dictionaries, lists, and tuples.

In [1]:
import math
import re

def clean(doc:str) -> str:
    """
    Cleans a string for processing.
    
    Removes non-alphanumeric characters and extra whitespace.
    Converts to lowercase.

    Parameters:
    doc (str): String to be cleaned

    Returns:
    str: Cleaned string
    """
    doc = re.sub(r'[^a-zA-Z0-9 ]', '', doc)  # remove non-alphanumeric or space characters
    doc = re.sub(r'\s+', ' ', doc)  # trim continous whitespace down to 1 space
    doc = doc.lower().strip()
    return doc

STOP_WORDS = {'a', 'an', 'the', 'of', 'for', 'and', 'or'}
def tokenize(doc:str) -> list:
    """
    Creates tokenized representation of a document string.

    Splits a string on whitespace, ignoring words that are in `STOP_WORDS`.

    Parameters:
    doc (str): String to be tokenized

    Returns:
    [str]: list of individual words
    """
    tokens = [word for word in doc.split() if word not in STOP_WORDS]
    return tokens

def frequency(tokens:list, seq_len:int = 5) -> dict:
    """
    Counts the frequency of unique sequences in a list of tokens.

    Returns a dictionary where keys are unique sequences of length 1 to `seq_len`
    and values are the count of occurences of those sequences in the `tokens` list.

    Parameters:
    tokens (list): list of tokens parsed from a document.
    seq_len (int): (min 1) max length of sequences to count.

    Returns:
    dict: {sequence: count}
    """
    assert seq_len >= 1, 'seq_len must be at least 1.'
    seq_count = {}

    for length in range(1, seq_len + 1):
        for i in range(len(tokens) - (length - 1)):
            seq = tuple(tokens[i:i+length])
            if seq in seq_count:
                seq_count[seq] = seq_count[seq] + 1
            else:
                seq_count[seq] = 1

    return seq_count

# took these two functions from
# https://www.geeksforgeeks.org/measuring-the-document-similarity-in-python/
# modified `vector_angle` from above into `cosine_similarity` based on
# https://neo4j.com/docs/graph-algorithms/current/labs-algorithms/cosine/
# because the results were not making sense to me
# (comparing a document to itself produced a score of 0.0)
def dot_product(tf1, tf2):
    """
    Calculates the dot product of two term frquency vectors.

    Returns the dot product of the frequencies of matching terms
    in two term frequency vectors. The term frequency vectors are
    dictionaries with the term as the key and the frequency as the value.

    Parameters:
    tf1 (dict): Term frequency vector 1 {(term, ): frequency}
    tf2 (dict): Term frequency vector 2 {(term, ): frequency}

    Returns:
    int: Dot product of the frequencies of matching terms
    """
    sum = 0.0
    for k1 in tf1:
        if k1 in tf2:
            sum = sum + (tf1[k1] * tf2[k1])
    return sum

# def vector_angle(tf1, tf2):
def cosine_similarity(tf1, tf2):
    """
    Calculates the cosine similarity between two term frequency vectors.

    Returns the cosine similarity of two term frequency vectors. The term
    frequency vectors are dictionaries with the term as the key and the
    frequency as the value.

    Parameters:
    tf1 (dict): Term frequency vector 1 {(term, ): frequency}
    tf2 (dict): Term frequency vector 2 {(term, ): frequency}

    Returns:
    float: Cosine similarity of two term frequency vectors
    """
    numerator = dot_product(tf1, tf2)
    denominator = math.sqrt(dot_product(tf1, tf1)) * math.sqrt(dot_product(tf2, tf2))
      
    return numerator / denominator

In [2]:
def similarity(d1, d2):
    """
    Calculate the similarity between two document strings.

    Returns the rounded cosine similarity between two documents
    after cleaning and tokenizing them.

    Parameters:
    d1 (str): Document 1
    d2 (str): Document 2

    Returns:
    float: similarity
    """
    # clean and tokenize
    d1 = tokenize(clean(d1))
    d2 = tokenize(clean(d2))

    # build token index so as to not be operating on strings
    vocab = list(set(d1 + d2))
    v1 = [vocab.index(token) for token in d1]
    v2 = [vocab.index(token) for token in d2]

    # calculate token frequency
    # single tokens gives best results on sample documents
    # tested sequences from 1 to 5 length
    seq_length = 1
    f1 = frequency(v1, seq_length)
    f2 = frequency(v2, seq_length)

    return round(cosine_similarity(f1, f2), 2)

In [3]:
doc1 = "The easiest way to earn points with Fetch Rewards is to just shop for the products you already love. If you have any participating brands on your receipt, you'll get points based on the cost of the products. You don't need to clip any coupons or scan individual barcodes. Just scan each grocery receipt after you shop and we'll find the savings for you."

doc2 = "The easiest way to earn points with Fetch Rewards is to just shop for the items you already buy. If you have any eligible brands on your receipt, you will get points based on the total cost of the products. You do not need to cut out any coupons or scan individual UPCs. Just scan your receipt after you check out and we will find the savings for you."

doc3 = "We are always looking for opportunities for you to earn more points, which is why we also give you a selection of Special Offers. These Special Offers are opportunities to earn bonus points on top of the regular points you earn every time you purchase a participating brand. No need to pre-select these offers, we'll give you the points whether or not you knew about the offer. We just think it is easier that way."

In [4]:
print('1,2', similarity(doc1, doc2))
print('1,3', similarity(doc1, doc3))
print('2,3', similarity(doc2, doc3))

1,2 0.85
1,3 0.51
2,3 0.53
